# Sistema criptográfico RSA

Ricardo Silva a71532

Francisca Fernandes a72450

João Cerqueira a65432

José Barbosa a69136

# Introdução

Com o crescimento da internet e a necessidade de transferencia de dados com segurança, começaram a ser desenvolvidos algoritmos criptográficos para assegurar a integridade dos dados, assim assegurando uma comunicação segura entre as duas partes, geralmente a estrutura cliente/servidor. O algoritmo RSA foi descrito em 1977 por Ron Rivest, Adi Shamir, e Leonard Adleman.

Este tipo de algoritmo é chamado de criptografia assimétrica ou de chave pública, pois existem duas chaves que são usadas.

É uma ideia simples mas que vem revolucionar a criptografia. Quem quiser comunicar de forma segura, encripta a mensagem com a chave pública do receptor e só ele consegue desencriptá-la com a respectiva chave privada, que não precisa nem deve ser partilhada com ninguém. Para além de estabelecer comunicações seguras, os sistemas criptográficos de chave pública têm por objectivo autenticar através de uma assinatura digital a informação transmitida.

Para implementar um sistema de assinaturas digitais com RSA, o utilizador que possua uma chave privada d poderá assinar uma dada mensagem como iremos abordar mais a baixo.

# Geração de chaves 

A chave pública é o par (n,e).

A chave privada é d.

Vamos supor que um cliente querer enviar uma mensagem a um servidor:

Sejam p,q dois numeros primos aleatorios:

In [62]:
p = random_prime(1000, lbound=150)
q = random_prime(1000, lbound=150)
p, q

(379, 181)

Seja n=pq, entao φ(n)=φ(pq)=(p−1)(q−1) pois p,q sao primos.

$e\in \mathbb{Z}_{\varphi(n)}^*$, ou seja, $e\in \mathbb{Z}_{\varphi(n)}$ com $(e, \varphi(n))=1$. Calcula $d = e^{-1}$ em $\mathbb{Z}_{\varphi(n)}$.
​	 


In [63]:
n= p*q
m =(p-1)*(q-1)
Zn=IntegerModRing(n)
Zm=IntegerModRing(m)

In [64]:
e = 19
gcd(e, m)


1

In [65]:
n

68599

Então o par(598901,19) seria a chave publica do servidor.

A sua chave privada vai ser calculada a partir do e. 

In [66]:
d = power_mod(e, -1, m)
d

64459

In [67]:
ChPub = (n,e)
ChPriv = d

Possivel algoritmo para fazer esta geração de chaves seria:

In [2]:
def RSA_Key(nbits = 512):
    p = random_prime(2^(nbits//2), 2^(nbits//2-1))
    q = random_prime(2^(nbits//2+1), 2^(nbits//2))
    n = p*q
    m = (p-1)*(q-1)
    e = randint(2, m-1)
    while gcd(e, m) != 1:
        e = randint(2, m-1)
    d = power_mod(e, -1, m)
    PubKey = (n, e)
    PrivKey = d
    return PubKey, PrivKey

# Cifração

Seja x a mensagem que o cliente quer enviar então , essa mensgaem vai ser cifrada numa mensagem y antes de ser enviada, e depois sera decifrada pelo servidor.

In [69]:
x = 123
y = power_mod(x, e, n)
y

65424

Um possivel Algoritmo para a cifração :

In [70]:
def RSA_encrypt(x, chave_pub):
    n, e = chave_pub
    criptograma = power_mod(x, e, n)
    return criptograma

# Decifração
O servidor vai decifrar a mensagem y e voltar a obter a mensagem x original.

In [71]:
power_mod(y, d, n)

123

Um possivel Algoritmo para a decifração :

In [72]:
def RSA_decrypt(y, chave):
    n, _ = chave[0]
    d = chave[1]
    z = power_mod(y, d, n)
    return z

# Assinatura Digital
Ao enviar uma mensagem, podemos também querer assiná-la, de forma semelhante ao que faríamos com um documento físico. 
Através do sistema RSA, é possível assinar uma mensagem de acordo com o conceito de assinatura com recuperação de mensagens. Ou seja, podemos assinar a mensagem de forma a que o remetente obtenha o seu conteúdo através da verificação da assinatura.

Esta assinatura digital não é mais do que um inteiro que garante a origem e integridade da mensagem.

Assinatura RSA funciona em 3 passos:

 - **Geração de chaves** - De forma semelhante à cifração de mensagens, o remetente começa por gerar a chave publica _(e,n)_ e a chave privada _d_, em que $n = pq$ é o produto de dois primos muito grandes.
 

 - **Geração da assinatura** - após codificar a mensagem _m_ em $\mathbb {Z}/n\mathbb {Z}$, é gerada a assinatura $s = m^d$ através da chave privada *d*.
 

 - **Verificação** - Computar $m = s^e \in \mathbb {Z}/n\mathbb {Z}$.



In [19]:
# 1º passo - geração de chaves
ChPub, ChPriv = RSA_Key(64)
n = ChPub[0] # n = p*q
Zn = IntegerModRing(n) # Zn = Z/nZ
print("Chave publica:", ChPub, "\n")
print("Chave privada:", ChPriv)

Chave publica: (7129612338089899369, 5439790653683557309) 

Chave privada: 3243342941345494109


In [20]:
# 2º passo - geração da assinatura
# Seja 'm' a mensagem a enviar
m = Zn(1234)
m

1234

In [21]:
# geramos a assinatura a partir da mensagem e da chave privada
d = ChPriv
s = m**d
s

6616111484528945985

In [22]:
# 3º passo - Verificação
e = ChPub[1]
m_decoded = s^e
m_decoded

1234

In [23]:
# gerar assinatura
S = power_mod(m, d, n) # Decifração da mensagem para gerar assinatura
S

6616111484528945985

In [24]:
# verificação
M_received = power_mod(S, e, n) # Cifração da mensagem para verificar origem
M_received

1234